In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms
import torch
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import ToPILImage
from models import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
class rellis_dataset(Dataset):
    """
    Using 6 classes
    """
    def __init__(self, data_root_dir, txt_file, transform=None, target_transform=None):
        """
        data_root_dit: root directory of the dataset, e.g. './data/rellis'
        txt_file: path to the train.txt listing image files
        
        """
        self.data_root_dir = data_root_dir
        self.transform = transform
        self.target_transform = target_transform
        with open(txt_file, 'r') as f:
            self.image_paths = [line.strip() for line in f.readlines()]
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.data_root_dir, 'image', self.image_paths[index] + '.jpg')
        ann_path = os.path.join(self.data_root_dir, 'annotation', self.image_paths[index] + '_group6.png')
        image = Image.open(img_path).convert('RGB')
        annotation = Image.open(ann_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            annotation = self.target_transform(annotation)
        annotation = (annotation*256).long() # turn the label to the interger
        return image, annotation

In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to desired size
    transforms.ToTensor(),
])

target_transform = transforms.Compose([
    transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.NEAREST),
    transforms.ToTensor()
]) 

val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to the same size as training
    transforms.ToTensor(),
])
train_dataset = rellis_dataset(data_root_dir='./data/rellis', txt_file='./data/rellis/train.txt', transform=train_transforms, target_transform=target_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

val_dataset = rellis_dataset(data_root_dir='./data/rellis', txt_file='./data/rellis/val.txt', transform=val_transforms,target_transform=target_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [5]:
model = make_SegFormerB1(num_classes=6)
model = model.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [7]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    total_iterations = 0  
    for images, masks in tqdm(loader, desc="Training"):
        images = images.to(device)
        masks = masks.squeeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        total_iterations += 1  

    
        if total_iterations % 10 == 0:
            avg_loss = running_loss / (total_iterations * loader.batch_size)
            print(f"Iteration {total_iterations}: Training Loss: {avg_loss:.4f}")

    return running_loss / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    total_iterations = 0 

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Validation"):
            images = images.to(device)
            masks = masks.squeeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)

            running_loss += loss.item() * images.size(0)
            total_iterations += 1  

         
            if total_iterations % 10 == 0:
                avg_loss = running_loss / (total_iterations * loader.batch_size)
                print(f"Iteration {total_iterations}: Validation Loss: {avg_loss:.4f}")

    return running_loss / len(loader.dataset)


In [8]:
num_epochs = 25
best_val_loss = np.inf

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    
    scheduler.step()

    print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Save the best model
    if val_loss < best_val_loss:
        print("Saving best model...")
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_segmentation_model.pth')

Epoch 1/25


Training:  10%|▉         | 10/104 [00:06<00:49,  1.92it/s]

Iteration 10: Training Loss: 1.4177


Training:  19%|█▉        | 20/104 [00:11<00:42,  1.98it/s]

Iteration 20: Training Loss: 1.2181


...

Iteration 10: Validation Loss: 0.6019


Validation:  65%|██████▍   | 20/31 [00:04<00:01,  5.94it/s]

Iteration 20: Validation Loss: 0.5581


Validation: 100%|██████████| 31/31 [00:06<00:00,  4.73it/s]

Iteration 30: Validation Loss: 0.6425
Train Loss: 0.2155, Val Loss: 0.6358
